<a href="https://colab.research.google.com/github/EmreYY20/ToS-Simplification/blob/dev/Emre_Summarization_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Imports